In [34]:
import pandas as pd
import ast

In [40]:
df=pd.read_excel("jaipur_cars.xlsx")


In [41]:

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])


In [42]:
new_car_details_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,1487,Asta 1.4 CRDi,₹ 5.10 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,10403,VXI,₹ 3.81 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,3982,VXI AT,₹ 3.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,5602,1.2 Kappa Magna BSIV,₹ 4.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,4319,AMT VXI,₹ 3.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,438,LT,₹ 1.08 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,3917,STD,₹ 1.05 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,8350,RXZ,₹ 8.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,3904,Std,₹ 1 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


In [19]:


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)


In [20]:
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
result=[]
for sec in df["new_car_feature"]:
   
   
    df1 = pd.DataFrame([item['value'] for item in sec["top"]], columns=['Features'])

# Concatenate all the values into a single string and place it in a single row
    concatenated_features = ','.join(df1['Features'])
    result.append({'Features': concatenated_features})

df_car_fearures = pd.DataFrame(result)

In [21]:
df['new_car_specs'] = df['new_car_specs'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def extract_specs(specs):
    """
    Extracts key-value pairs from both 'top' and 'list' fields in the specs dictionary.
    """
    # Extract from 'top'
    top_data = extract_keys(specs.get('top', []))
    
    # Extract from 'list' in 'data'
    list_data = {}
    if 'data' in specs:
        for section in specs['data']:
            list_data.update(extract_keys(section.get('list', [])))
    
    # Combine both dictionaries
    combined_data = {**top_data, **list_data}
    return combined_data

# Apply the function to each row in the 'new_car_specs' column
car_spec_df = df["new_car_specs"].apply(extract_specs).apply(pd.Series)

# Display the resulting DataFrame


In [22]:
new_car_details_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,1487,Asta 1.4 CRDi,₹ 5.10 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,10403,VXI,₹ 3.81 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,3982,VXI AT,₹ 3.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,5602,1.2 Kappa Magna BSIV,₹ 4.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,4319,AMT VXI,₹ 3.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,438,LT,₹ 1.08 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,3917,STD,₹ 1.05 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,8350,RXZ,₹ 8.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,3904,Std,₹ 1 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days


In [43]:
combined_df = pd.concat([new_car_details_df,car_overview, df_car_fearures,car_spec_df ], axis=1)

In [46]:
combined_df

,state,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Top Speed,Acceleration,Tyre Type,Alloy Wheel Size,No Door Numbers,Cargo Volumn,Gross Weight,BoreX Stroke,Compression Ratio,Ground Clearance Unladen
0,jaipur,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,...,180 Kmph,11.9 Seconds,Tubeless,16,5,295-litres,NaN,NaN,NaN,NaN
1,jaipur,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,...,NaN,NaN,"Radial, Tubeless",NaN,5,268,1335,NaN,NaN,NaN
2,jaipur,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,...,150 Kmph,15.05 Seconds,"Tubeless, Radial",NaN,5,235-litres,1250kg,73 X 82 mm,11.0:1,NaN
3,jaipur,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,...,165 Kmph,12.9 Seconds,Tubeless,NaN,5,256,NaN,NaN,NaN,NaN
4,jaipur,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,...,152 kmph,18.6 Seconds,Tubeless Tyres,NaN,5,180-liters,1350kg,69 x 72 mm,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,jaipur,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,...,NaN,NaN,"Tubeless,Radial",14,5,170-litres,NaN,NaN,NaN,NaN
1116,jaipur,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,...,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,1140kg,68.5 x 72.0 mm,NaN,NaN
1117,jaipur,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,...,NaN,NaN,"Tubeless, Radial",16,5,405,NaN,NaN,NaN,NaN
1118,jaipur,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,...,137km/hr,17.7 seconds,"Tubeless,Radial",12,5,NaN,1140kg,68.5 x 72.0 mm,NaN,NaN


In [49]:
combined_df.insert(0,"state","jaipur")

ValueError: cannot insert state, already exists

In [50]:
combined_df.describe()

,it,ownerNo,modelYear,centralVariantId,Year of Manufacture,No of Cylinder,Values per Cylinder
count,1120.0,1120.000000,1120.000000,1120.000000,1114.000000,1113.000000,1113.000000
mean,0.0,1.499107,2015.771429,4663.301786,2015.857271,3.741240,3.877808
std,0.0,0.794305,4.357308,2572.664753,4.207586,0.517219,0.442139
min,0.0,0.000000,1998.000000,30.000000,2002.000000,2.000000,2.000000
25%,0.0,1.000000,2013.000000,2000.000000,2013.000000,3.000000,4.000000
50%,0.0,1.000000,2016.000000,4310.000000,2016.000000,4.000000,4.000000
75%,0.0,2.000000,2019.000000,6938.250000,2019.000000,4.000000,4.000000
max,0.0,5.000000,2023.000000,11052.000000,2023.000000,8.000000,4.000000
